# Notebook for training tensorflow object detector API

## Downloads SSDLite Mobiledet model

In [1]:
!wget http://download.tensorflow.org/models/object_detection/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz -P pretrained_models/
!tar xf pretrained_models/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz --directory pretrained_models/
!cp /home/mikkel/models/research/object_detection/samples/configs/ssdlite_mobiledet_edgetpu_320x320_coco_sync_4x4.config /home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/pretrained_models/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/ssdlite_mobiledet_edgetpu_320x320_coco_sync_4x4.config

--2020-12-01 17:20:34--  http://download.tensorflow.org/models/object_detection/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.74.48, 2a00:1450:400f:801::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.74.48|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156413934 (149M) [application/x-tar]
Saving to: ‘pretrained_models/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz’

ssdlite_mobiledet_e 100%[===================>] 149.17M  26.9MB/s    in 5.5s    

2020-12-01 17:20:41 (26.9 MB/s) - ‘pretrained_models/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz’ saved [156413934/156413934]



## Dataset

In [2]:
dataset_path = '/home/mikkel/Documents/experts_in_teams_proj/vision/data/tf_hole_fence/TFRecords/'

## Edit pipeline config to configure the training

In [3]:
import tensorflow as tf
print("Tensorflow version", tf.__version__)
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Tensorflow version 1.15.0
Tue Dec  1 17:20:46 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 455.45.01    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1070    On   | 00000000:09:00.0 Off |                  N/A |
| 47%   59C    P0    58W / 151W |    328MiB /  8119MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                              

In [4]:
pipeline = pipeline_pb2.TrainEvalPipelineConfig()
config_path = '/home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/pretrained_models/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/ssdlite_mobiledet_edgetpu_320x320_coco_sync_4x4.config'#'/home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/pretrained_models/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/ssd_mobilenet_v2_quantized_300x300_coco.config'#
# Read config file and converts it to editable pipeline
with tf.gfile.GFile(config_path, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline)
    

pipeline.train_input_reader.tf_record_input_reader.input_path[:] = [dataset_path + 'train/hole.tfrecord']
pipeline.train_input_reader.label_map_path = dataset_path + 'train/hole_label_map.pbtxt'
pipeline.eval_input_reader[0].tf_record_input_reader.input_path[:] = [dataset_path + 'valid/hole.tfrecord']
pipeline.eval_input_reader[0].label_map_path = dataset_path + 'valid/hole_label_map.pbtxt'
pipeline.train_config.fine_tune_checkpoint = '/home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/pretrained_models/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/model.ckpt'
pipeline.train_config.batch_size = 16
pipeline.train_config.num_steps = 50000
pipeline.model.ssd.num_classes = 1
# Enable ssdlite, this should already be enabled in the config we downloaded, but this is just to make sure.
pipeline.model.ssd.box_predictor.convolutional_box_predictor.kernel_size = 3
pipeline.model.ssd.box_predictor.convolutional_box_predictor.use_depthwise = True
pipeline.model.ssd.feature_extractor.use_depthwise = True
# Quantization Aware Traning IMPORTANT!!!!
pipeline.graph_rewriter.quantization.delay = 0
pipeline.graph_rewriter.quantization.weight_bits = 8
pipeline.graph_rewriter.quantization.activation_bits = 8

config_text = text_format.MessageToString(pipeline) # Back to text to write
with tf.gfile.Open(config_path, "wb") as f:
    f.write(config_text)

# This is out config after modifying
!cat '/home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/pretrained_models/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/ssdlite_mobiledet_edgetpu_320x320_coco_sync_4x4.config'

model {
  ssd {
    num_classes: 1
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobiledet_edgetpu"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 4e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.03
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.97
          center: true
          scale: true
          epsilon: 0.001
          train: true
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: false
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        un

## Tensorboard

In [4]:
%load_ext tensorboard
%tensorboard --logdir trained_models/

Reusing TensorBoard on port 6006 (pid 20525), started 0:48:14 ago. (Use '!kill 20525' to kill it.)

## Training

In [ ]:
!python model_main.py \
        --logtostderr=true \
        --model_dir='/home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/trained_models/my_ssdlite_mobiledet_edgetpu_320x320' \
        --pipeline_config_path='/home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/pretrained_models/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/ssdlite_mobiledet_edgetpu_320x320_coco_sync_4x4.config'

W1201 17:20:52.882973 139737518040896 model_lib.py:793] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: None
I1201 17:20:52.883114 139737518040896 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1201 17:20:52.883181 139737518040896 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I1201 17:20:52.883244 139737518040896 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I1201 17:20:52.883306 139737518040896 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W1201 17:20:52.883382 139737518040896 model_lib.py:809] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to_t

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1201 17:21:03.256890 139737518040896 deprecation.py:323] From /home/mikkel/anaconda3/envs/tf1/lib/python3.6/site-packages/object_detection/inputs.py:109: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W1201 17:21:03.423949 139737518040896 deprecation.py:323] From /home/mikkel/anaconda3/envs/tf1/lib/python3.6/site-packages/object_detection/inputs.py:93: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 inst

2020-12-01 17:21:46.253088: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-01 17:21:46.254353: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1304] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 7316 MB memory) -> physical GPU (device: 0, name: GeForce GTX 1070, pci bus id: 0000:09:00.0, compute capability: 6.1)
2020-12-01 17:21:46.257179: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5647ea8d8fb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2020-12-01 17:21:46.257201: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): GeForce GTX 1070, Compute Capability 6.1
INFO:tensorflow:Running local_init_op.
I1201 17:21:53.665046 139737518040896 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
W1201 17:32:18.821379 139737518040896 deprecation.py:323] From /home/mikkel/anaconda3/envs/tf1/lib/python3.6/site-packages/object_detection/utils/visualization_utils.py:618: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecate

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1167: /home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/trained_models/my_ssdlite_mobiledet_edgetpu_320x320/model.ckpt-1167
I1201 17:32:32.060302 139737518040896 estimator.py:2109] Saving 'checkpoint_path' summary for global step 1167: /home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/trained_models/my_ssdlite_mobiledet_edgetpu_320x320/model.ckpt-1167
INFO:tensorflow:global_step/sec: 1.40835
I1201 17:32:49.354751 139737518040896 basic_session_run_hooks.py:692] global_step/sec: 1.40835
INFO:tensorflow:loss = 3.7023084, step = 1200 (71.005 sec)
I1201 17:32:49.357300 139737518040896 basic_session_run_hooks.py:260] loss = 3.7023084, step = 1200 (71.005 sec)
INFO:tensorflow:global_step/sec: 1.97462
I1201 17:33:39.997272 139737518040896 basic_session_run_hooks.py:692] global_step/sec: 1.97462
INFO:tensorflow:loss = 3.6614246, step = 1300 (50.642 sec)
I1201 17:33:39.998795 139737518040

...

INFO:tensorflow:Done calling model_fn.
I1202 00:02:28.269677 139737518040896 estimator.py:1150] Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-12-02T00:02:28Z
I1202 00:02:28.287818 139737518040896 evaluation.py:255] Starting evaluation at 2020-12-02T00:02:28Z
INFO:tensorflow:Graph was finalized.
I1202 00:02:29.121000 139737518040896 monitored_session.py:240] Graph was finalized.
2020-12-02 00:02:29.121435: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-02 00:02:29.121846: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1070 major: 6 minor: 1 memoryClockRate(GHz): 1.721
pciBusID: 0000:09:00.0
2020-12-02 00:02:29.121926: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-12-02 00:02:29.12

INFO:tensorflow:global_step/sec: 2.07557
I1202 00:04:15.133387 139737518040896 basic_session_run_hooks.py:692] global_step/sec: 2.07557
INFO:tensorflow:loss = 0.4650048, step = 48800 (48.179 sec)
I1202 00:04:15.136178 139737518040896 basic_session_run_hooks.py:260] loss = 0.4650048, step = 48800 (48.179 sec)
INFO:tensorflow:global_step/sec: 2.07982
I1202 00:05:03.214422 139737518040896 basic_session_run_hooks.py:692] global_step/sec: 2.07982
INFO:tensorflow:loss = 0.40325624, step = 48900 (48.081 sec)
I1202 00:05:03.216806 139737518040896 basic_session_run_hooks.py:260] loss = 0.40325624, step = 48900 (48.081 sec)
INFO:tensorflow:global_step/sec: 2.08331
I1202 00:05:51.214953 139737518040896 basic_session_run_hooks.py:692] global_step/sec: 2.08331
INFO:tensorflow:loss = 0.6441012, step = 49000 (48.001 sec)
I1202 00:05:51.217593 139737518040896 basic_session_run_hooks.py:260] loss = 0.6441012, step = 49000 (48.001 sec)
INFO:tensorflow:global_step/sec: 2.11974
I1202 00:06:38.390427 13973

INFO:tensorflow:Done calling model_fn.
I1202 00:12:29.084230 139737518040896 estimator.py:1150] Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-12-02T00:12:29Z
I1202 00:12:29.098683 139737518040896 evaluation.py:255] Starting evaluation at 2020-12-02T00:12:29Z
INFO:tensorflow:Graph was finalized.
I1202 00:12:29.897527 139737518040896 monitored_session.py:240] Graph was finalized.
2020-12-02 00:12:29.897994: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-02 00:12:29.898380: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1070 major: 6 minor: 1 memoryClockRate(GHz): 1.721
pciBusID: 0000:09:00.0
2020-12-02 00:12:29.898436: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-12-02 00:12:29.89

INFO:tensorflow:Saving checkpoints for 50000 into /home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/trained_models/my_ssdlite_mobiledet_edgetpu_320x320/model.ckpt.
I1202 00:14:10.486546 139737518040896 basic_session_run_hooks.py:606] Saving checkpoints for 50000 into /home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/trained_models/my_ssdlite_mobiledet_edgetpu_320x320/model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (600 secs).
I1202 00:14:12.198863 139737518040896 training.py:527] Skip the current checkpoint eval due to throttle secs (600 secs).
INFO:tensorflow:Reading unweighted datasets: ['/home/mikkel/Documents/experts_in_teams_proj/vision/data/tf_hole_fence/TFRecords/valid/hole.tfrecord']
I1202 00:14:12.223926 139737518040896 dataset_builder.py:148] Reading unweighted datasets: ['/home/mikkel/Documents/experts_in_teams_proj/vision/data/tf_hole_fence/TFRecords/valid/hole.tfrecord']
INFO:tensorflow:Reading r

INFO:tensorflow:Done calling model_fn.
I1202 00:14:18.453698 139737518040896 estimator.py:1150] Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-12-02T00:14:18Z
I1202 00:14:18.467999 139737518040896 evaluation.py:255] Starting evaluation at 2020-12-02T00:14:18Z
INFO:tensorflow:Graph was finalized.
I1202 00:14:19.263510 139737518040896 monitored_session.py:240] Graph was finalized.
2020-12-02 00:14:19.263950: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-02 00:14:19.264355: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1070 major: 6 minor: 1 memoryClockRate(GHz): 1.721
pciBusID: 0000:09:00.0
2020-12-02 00:14:19.264410: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-12-02 00:14:19.26

## Exports checkpoints to tflite graph and convert to .tflite model

In [ ]:
# exports checkpoints
!python export_tflite_ssd_graph.py \
        --pipeline_config_path=/home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/pretrained_models/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/ssdlite_mobiledet_edgetpu_320x320_coco_sync_4x4.config \
        --trained_checkpoint_prefix=/home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/trained_models/my_ssdlite_mobiledet_edgetpu_320x320/model.ckpt-50000 \
        --output_directory='/home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/output_models' \
        --add_postprocessing_op=true

# convert model to .tflite
!tflite_convert \
        --output_file="/home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/output_models/ssdlite_mobiledet_fence_hole.tflite" \
        --graph_def_file="/home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/output_models/tflite_graph.pb" \
        --inference_type=QUANTIZED_UINT8 \
        --input_arrays="normalized_input_image_tensor" \
        --output_arrays="TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3" \
        --mean_values=128 \
        --std_dev_values=128 \
        --input_shapes=1,320,320,3 \
        --allow_custom_ops

## Evaluating on coral accelerator

In [3]:
# Now we do evaluation on the tflite model.
import os
import numpy as np
from tflite_runtime.interpreter import Interpreter
from tflite_runtime.interpreter import load_delegate
from PIL import Image, ImageDraw, ImageFont
import time
%matplotlib inline

model_path = '/home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/output_models/ssdlite_mobiledet_fence_hole.tflite'
model_path_tpu = '/home/mikkel/Documents/experts_in_teams_proj/vision/tf_object_detection/output_models/ssdlite_mobiledet_fence_hole_edgetpu.tflite'

# Creates tflite interpreter
interpreter = Interpreter(model_path) 
print(interpreter)

# This exact code can be used to run inference on the edgetpu by simply creating 
# the instantialize the interpreter with libedgetpu delegates:
interpreter = Interpreter(model_path_tpu, experimental_delegates=[load_delegate('libedgetpu.so.1.0')])
interpreter.allocate_tensors()
interpreter.invoke() # warmup
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
width = input_details[0]['shape'][2]
height = input_details[0]['shape'][1]

def run_inference(interpreter, image):
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    boxes = interpreter.get_tensor(output_details[0]['index'])[0]
    classes = interpreter.get_tensor(output_details[1]['index'])[0]
    scores = interpreter.get_tensor(output_details[2]['index'])[0]
    # num_detections = interpreter.get_tensor(output_details[3]['index'])[0]
    return boxes, classes, scores

times = list()

test_path = '/home/mikkel/Documents/experts_in_teams_proj/vision/data/test/images'
test_image_paths = [f'{test_path}/{img}' for img in os.listdir(test_path)] 
for image_path in test_image_paths:
    image = Image.open(image_path)
    image_width, image_height = image.size
    draw = ImageDraw.Draw(image)
    resized_image = image.resize((width, height))
    np_image = np.asarray(resized_image)
    input_tensor = np.expand_dims(np_image, axis=0)
    
    # Run inference
    model_start = time.time()
    boxes, classes, scores = run_inference(interpreter, input_tensor)
    model_end = time.time() - model_start
    times.append(model_end)
    # Draw results on image
    colors = {0:(255, 0, 0)}
    labels = {0:'Hole'}
    for i in range(len(boxes)):
        if scores[i] > .5:
            ymin = int(max(1, (boxes[i][0] * image_height)))
            xmin = int(max(1, (boxes[i][1] * image_width)))
            ymax = int(min(image_height, (boxes[i][2] * image_height)))
            xmax = int(min(image_width, (boxes[i][3] * image_width)))
            draw.rectangle((xmin, ymin, xmax, ymax), width=12, outline=colors[int(classes[i])])
            draw.rectangle((xmin, ymin, xmax, ymin-10), fill=colors[int(classes[i])])
            text = labels[int(classes[i])] + ' ' + str(scores[i]*100) + '%'
            draw.text((xmin+2, ymin-10), text, fill=(0,0,0), width=2, font=ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 28))
    display(image)
    
times = np.array(times)
print(f'Took average pr images: {np.mean(times)} s')

ValueError: Failed to load delegate from libedgetpu.so.1.0


In [ ]:
!shutdown